In [1]:
import torch
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from utils import set_seed
from cnn_data_handler import get_dataloader
from model import CNN

torch.set_default_dtype(torch.float64)

ModuleNotFoundError: No module named 'torch'

In [2]:
set_seed(0)

NameError: name 'torch' is not defined

In [13]:
batch_size = 128
epochs = 1000

train_dataloader = get_dataloader('train', batch_size)
valid_dataloader = get_dataloader('valid', batch_size)


In [14]:
model = CNN()
model = model.cuda()

best_valid_loss = 100

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min = 0)


In [15]:
for epoch in tqdm(range(epochs)):
    running_loss = 0
    model.train()
    for data in train_dataloader:
        features, labels, n_interpolates = data
        features = torch.tensor(features).cuda()
        loss = None
        
        empty_start_arr = [] # -1이 시작되는 index를 caching
        
        # empty_start 찾는 for문
        for label in labels:

            if len(np.where(label.numpy().astype("int32") == -1)[0]) == 0:
                empty_start = 14
            else:
                empty_start = np.where(label.numpy().astype("int32") == -1)[0][0]
            
            empty_start_arr.append(empty_start)

        labels = torch.tensor(labels).cuda() # batchsize * 14

        outputs = model(features) # batchsize * 14

        optimizer.zero_grad()
        for i in range(len(outputs)):
            output = outputs[i] # shape : 14
            label = labels[i] # shape : 14
            n_interpolate = n_interpolates[i] # scalar

            

            empty_start = empty_start_arr[i] # ith 해당하는 empty start

            
            weight = torch.ones(empty_start) # shape [empty_start]
            weight *= (13.0 - n_interpolate) / 91 # 1 ~ 13 sum 
            weight[0:5] *= 15 
            weight = weight.cuda()

            with torch.no_grad():
                label = label[:empty_start] 

            if loss is None:
                # MSELoss(reduction=none) -> [empty_start] * weight
                loss = torch.mean(weight * nn.MSELoss(reduction='none')(output[:empty_start], label))
            else:
                loss += torch.mean(weight * nn.MSELoss(reduction='none')(output[:empty_start], label))



        running_loss += loss.item()

        loss /= len(outputs)

        loss.backward()

        optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, train running loss", running_loss / len(train_dataloader.dataset))

    valid_loss = 0
    model.eval()
    with torch.no_grad():
        for data in valid_dataloader:
            features, labels, n_interpolates = data
            features = torch.tensor(features).cuda()
            
            loss = None
            outputs = model(features)

            empty_start_arr = []
            for label in labels:
                if len(np.where(label.numpy().astype("int32") == -1)[0]) == 0:
                    empty_start = 14
                else:
                    empty_start = np.where(label.numpy().astype("int32") == -1)[0][0]
                empty_start_arr.append(empty_start)

            labels = torch.tensor(labels).cuda()

            for i in range(len(outputs)):
                output = outputs[i]
                label = labels[i]
                n_interpolate = n_interpolates[i]

                output = torch.clamp(output, min = 0)
                # weight = (13.0 - n_interpolate) / 91

                # # weight = torch.exp((13.0 - n_interpolate) / 91)
                # weight = weight.cuda()
            
                empty_start = empty_start_arr[i]
                with torch.no_grad():
                    label = label[:empty_start]
                    # label = torch.cat((label[0].unsqueeze(0), torch.diff(label)))
            

                if loss is None:
                    loss = nn.MSELoss()(output[:empty_start], label)
                else:
                    loss += nn.MSELoss()(output[:empty_start], label)


            valid_loss += loss.item()

        if valid_loss / len(valid_dataloader.dataset) < best_valid_loss:
            best_valid_loss = valid_loss / len(valid_dataloader.dataset)
            torch.save(model.state_dict(), "./best_model.pt")
            print("Best Valid loss: ", best_valid_loss)

        if epoch % 10 == 0:
            print(f"Epoch: {epoch}, valid running loss", valid_loss / len(valid_dataloader.dataset))

  0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_27700\4036381028.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(features).cuda()
C:\Users\user\AppData\Local\Temp\ipykernel_27700\4036381028.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).cuda() # batchsize * 14


Epoch: 0, train running loss 118.33827596911799


C:\Users\user\AppData\Local\Temp\ipykernel_27700\4036381028.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(features).cuda()
C:\Users\user\AppData\Local\Temp\ipykernel_27700\4036381028.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).cuda()
  0%|          | 1/1000 [00:05<1:23:38,  5.02s/it]

Best Valid loss:  13.852670582093259
Epoch: 0, valid running loss 13.852670582093259


  0%|          | 3/1000 [00:15<1:23:22,  5.02s/it]

...

 99%|█████████▉| 990/1000 [1:17:43<00:46,  4.60s/it]

Epoch: 990, train running loss 5.68311221824794


 99%|█████████▉| 991/1000 [1:17:48<00:41,  4.60s/it]

Epoch: 990, valid running loss 9.068736698517089


100%|██████████| 1000/1000 [1:18:30<00:00,  4.71s/it]


In [16]:
print(best_valid_loss)

3.8907197439834103
